### Presentación del proyecto

##**Objetivo**

Los aerolíneas juegan un rol muy importante en los movimientos de viaje y relaciones a nivel global, tanto por motivos laborales o de ocio.

Las reseñas de éstas, se convirtieron en un estándar para orientar a los usuarios en la comparación al momento de elegir. Pero además, permiten a las aerolíneas, obtener un insight sobre la experiencia de los pasajeros, sus preferencias y el nivel de satisfacción en cada caso.
<br><br>


Con el acceso a herramientas e información que poseemos hoy en día, podemos categorizar miles de reseñas en etiquetas de sentimiento positivo, negativo, o neutral.

El objetivo es trabajar este conjunto de datos con reseñas en texto de diversos hoteles, junto a una calificación de 1 a 5 (1 siendo muy negativo y 5 siendo muy positivo). A partir de esta clasificación, al obtener una reseña nueva, se puede predecir si ésta es positiva o negativa.

###Contenido del dataset

####Columnas

**Airline Name** = Nombre de la Aerolínea

**Overall Rating** = Calificación total

**Review Title** = Titulo de la reseña

**Review Date** = Fecha de la reseña

**Verified** = Si la reseña está verificada o no

**Review** = Texto de la reseña

**Aircraft** = Tipo de avión

**Type of Traveller** = Tipo de pasajero

**Seat Type** = Tipo de asiento

**Route** = Ruta de viaje

**Date Flown** = Fecha de vuelo

**Seat Comfort** = Calificación de comodidad del asiento

**Cabin Staff Service** = Calificación del staff de cabina

**Food & Beverages** = Calificación de la comida y bebida

**Ground Service** = Calificación del servicio en tierra

**Inflight Entertainment** = Calificación del entretenimiento en el vuelo

**Wifi & Connectivity** = Calificación de WiFi y conectividad

**Value for Money** = Calificación de relación precio/calidad

**Recommended** = Recomendado si/no


####Citaciones

Recopilación por Juhi Bhojani y subido en Kaggle y Github

https://www.kaggle.com/datasets/juhibhojani/airline-reviews

https://github.com/Juhibhojani/Airline-Reviews-

Las reviews de aerolíneas de este dataset fueron obtenidas por el autor desde:

https://www.airlinequality.com/review-pages/a-z-airline-reviews/

### **Lectura de datos**

**Librerias necesarias**

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
from collections import Counter
import re

import matplotlib.pyplot as plt
from wordcloud import WordCloud
import seaborn as sns

import spacy

#TF-IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

#Modelos
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from textblob import TextBlob

#NLTK
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.util import ngrams
import string

nltk.download('punkt')  # Tokenizers
nltk.download('punkt_tab')
nltk.download('stopwords')  # Stop words
nltk.download('wordnet')  # WordNet lemmatizer
nltk.download('averaged_perceptron_tagger_eng')

In [ ]:
url = "https://raw.githubusercontent.com/Larrentawn/NLP-project/refs/heads/main/Airline_review.csv"
df = pd.read_csv(url, sep=",")

In [ ]:
nlp = spacy.load("en_core_web_sm")

##**Análisis inicial**

In [ ]:
df.head()

Podemos observar algunos de los valores que tenemos en el dataset, asi como sus columnas. Algo que podemos ver a simple vista es que las fechas estan en formato escrito.

In [ ]:
df.shape

In [ ]:
df.describe()

Vemos datos como la media y la desviación estandar.

In [ ]:
df.dtypes

Podemos ver varias cosas, los nombres de las columnas tienen mayúsculas, símbolos y espacios.

La columna **Overall Rating** es de tipo **object** en lugar de **int** o **float** y los de columna temporal que nos interesa **(Review Date)** es también de tipo object en lugar de **datetime**. También la variable de **Recommended** que es si/no.

In [ ]:
pd.set_option('display.max_colwidth', None)
df.describe(include="O").T

Se puede ver más en profundidad las columnas de tipo **Object**, un conteo de valores unicos, el top y su frecuencia.

Vemos que las calificaciones **(overall_rating)** casi la mitad de los valores son 1, es decir posee muchisimas calificaciones negativas.

Las variables **type_of_traveller**, **seat_type** y **recommended** podrian ser codificadas en categorias numericas.
Tambien con la variable **overall_rating**.

Se procede a normalizar los nombres de las columnas

In [ ]:
df.columns = [col.replace(" ", "_").lower() for col in df.columns]
df.dtypes

In [ ]:
df['overall_rating'].unique()

En las calificaciones encontramos el valor 'n', el cual probablemente sea null.
Podemos ver un aproximado de donde podria ser alineada esta no-calificacion con algunos indicadores.


In [ ]:
#Podemos leer algunas de las reseñas con este valor 'n'

df[df['overall_rating'] == 'n'][['review', 'overall_rating']].sample(5)

Tambien podemos ver si respondió si recomendaría o no la aerolinea

In [ ]:
#Se filtra por respuesta 'n' en rating
df_overall_rating_n = df[df['overall_rating'] == 'n']

#Visualizacion de respuesta de recomienda si o no
plt.figure(figsize=(6,4))
sns.countplot(data=df_overall_rating_n, x='recommended')
plt.title('Recommend (Sí/No) cuando overall_rating = "n"')
plt.xlabel('Recommend')
plt.ylabel('Cantidad')
plt.show()


Vemos entonces que la mayoria de las 'n' serian tambien calificaciones negativas (1, 2, 3)

Procedemos a reemplazar 'n' con la mediana.

In [ ]:
#Convertimos en numérico
df['overall_rating'] = pd.to_numeric(df['overall_rating'], errors='coerce')

#Calculamos la mediana
overall_rating_median = df['overall_rating'].median()
print(overall_rating_median)

In [ ]:
#Reemplazamos los nulos con la mediana
df['overall_rating'].fillna(round(overall_rating_median), inplace=True)
df['overall_rating'] = df['overall_rating'].astype(int) #Convertimos a enteros

In [ ]:
#Verificamos los valores unicos
df['overall_rating'].unique()

Se eliminan columnas irrelevantes

In [ ]:
df.drop(["unnamed:_0", "aircraft","route","date_flown"], axis=1, inplace=True)

Condicionales para ayudar en el formateo de la columna de fecha

In [ ]:
df['review_date'] = [col.replace("nd","") if "nd" in col else col for col in df['review_date'].values]
df['review_date'] = [col.replace("th","") if "th" in col else col for col in df['review_date'].values]
df['review_date'] = [col.replace("st","") if "st" in col else col for col in df['review_date'].values]
df['review_date'] = [col.replace("rd","") if "rd" in col else col for col in df['review_date'].values]

df['review_date'] = [col.replace("August","Augu") if "August" in col else col for col in df['review_date'].values]
df['review_date'] = [col.replace("Augu","August") if "Augu" in col else col for col in df['review_date'].values]

df['review_date'] = pd.to_datetime(df['review_date'])
df['review_date']

In [ ]:
# Verificación de nulos
df.isnull().sum()

####Verificación de duplicados

In [ ]:
print(df.duplicated().sum())

In [ ]:
#Se eliminan duplicados y se vuelve a verificar
df.drop_duplicates(inplace=True)
print(df.duplicated().sum())

##EDA

In [ ]:
df.hist(figsize=[10,8], bins=14)

Vemos que hay una gran cantidad de reseñas fueron hechas en los ultimos años, y que hay muchas reseñas de 1 estrella en varias de las categorias.
La categoria de Staff y servicio de cabina es la que mas califcaciones positivas posee.

In [ ]:
#Distribucion de calificaciones
plt.figure(figsize=(4,3))
plt.title("Distribucion de calificaciones totales")
sns.set_style("darkgrid")
sns.countplot(df,x='overall_rating')

In [ ]:
plt.title("Calificacion de precio-calidad por calificacion total")
sns.set_style("darkgrid")
sns.violinplot(df, x='overall_rating', y='value_for_money')

Observamos correlación en que a mayor calificacion relacion calidad-precio, mayor calificacion total. Hay una tendencia clara.

In [ ]:
#Distribucion de recommended
plt.figure(figsize=(4,3))
plt.title("Distribucion de recomienda si/no")
sns.set_style("darkgrid")
sns.countplot(df,x='recommended')

In [ ]:
# Conversion de la columna Recommended de Yes/no a 1 y 0 respectivamente
df["recommended"] = df["recommended"].map(dict(yes=1, no=0))
print(df[["recommended"]].to_string())

In [ ]:
# Selección de columnas numéricas y de texto

df_num = df[["overall_rating", "review_date", "verified",
"seat_comfort", "cabin_staff_service", "food_&_beverages",
"ground_service", "inflight_entertainment", "wifi_&_connectivity", "value_for_money", "recommended"]]

df_text = df[["review", "type_of_traveller", "seat_type"]]

In [ ]:
print(df.columns)

In [ ]:
df['airline_name'].value_counts().reset_index()

In [ ]:
# Recuento de cantidad de reviews por aerolinea
df['airline_review_count'] = df.groupby('airline_name')['airline_name'].transform('count')
df[['airline_name', 'airline_review_count']].drop_duplicates().sort_values('airline_review_count', ascending=False)

In [ ]:
# Visualizacion del recuento de reviews por aerolinea (top 250)

df_airlines = df['airline_name'].value_counts().head(250).reset_index()
df_airlines.columns = ['airline_name', 'airline_review_count']

plt.figure(figsize=(25, 9))
sns.barplot(data=df_airlines, y='airline_review_count', x='airline_name', palette='viridis', hue='airline_name', legend=False)
plt.xticks(rotation=90, fontsize=9)

plt.title('Cantidad de Reseñas por Aerolínea')
plt.xlabel('Cantidad de Reseñas')
plt.ylabel('Aerolínea')
plt.tight_layout()
plt.show()

Vemos que las primeras 150 aerolineas poseen 100 reseñas cada una,, y luego va en disminución.

In [ ]:
df_name_rev_count = df.groupby("airline_name").mean(numeric_only=True).drop(columns=["airline_review_count"]).reset_index()
df_name_rev_count

In [ ]:
# merge de los dos dataframes con el recuento de aerolineas y los ratings
df_summ = pd.merge(
    df_airlines,
    df_name_rev_count,
    how="inner",
    on='airline_name')
df_summ.head()

In [ ]:
# Visualización de los ratings y la calificacion de precio-calidad
sns.set_theme(style="whitegrid")
plt.subplots(figsize=(8,6))
sns.scatterplot(df_summ, x="value_for_money", y="overall_rating",
                 size='airline_review_count', hue='verified')
plt.title("Promedio de rating por promedio de precio-calidad")
plt.xlabel('Valoración calidad-precio')
plt.ylabel('Calificación total')
plt.tight_layout()
plt.show()

Vemos en mayor detalle la correlación que existe entre la calificacion promedio de cada aerolinea y el promedio de calificacion calidad-precio.
La cantidad de reseñas se puede ver con el tamaño de cada circulo y el color indica cuantas de esas reseñas estan verificadas por el sitio.

In [ ]:
plt.title("Calificacion de precio-calidad por calificacion total")
sns.set_style("darkgrid")
sns.violinplot(df, y='overall_rating', x='recommended')

Vemos tambien una relacion entre la calificacion total y la recomendacion o no recomendacion de la aerolinea

In [ ]:
df_num = df_num.drop(columns=["review_date"])
print(df_num.columns)
print(df_num.columns.names)

In [ ]:
corr = df_num.corr()

used_columns = df_num.columns[df_num.columns.isin(corr.columns)]

df_corr = df_num[used_columns]
corr_matrix = df_corr.corr()

# Graficar con números
plt.figure(figsize=(8, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="vlag", center=0, linewidths=.75)


Vemos en la matriz de correlaciones que la mayor correlacion se puede encontrar entre value_for_money (precio-calidad), y recommended. Tambien entre otras calificaciones de servicio en aeropuerto y value_for_money.

# **Análisis y preprocesamiento del Texto**

####**Longitud de las Oraciones**

In [ ]:
print(df["review"].dropna().iloc[0])

In [ ]:
df["review"] = df["review"].fillna("")

def longitud_oraciones(text):
    try:
        oraciones = sent_tokenize(text)
        if not oraciones:
            return pd.Series([0, 0])
        len_oraciones = [len(word_tokenize(sent)) for sent in oraciones]
        return pd.Series([len(oraciones), sum(len_oraciones) / len(len_oraciones)])
    except:
        return pd.Series([0, 0])

# Aplicamos la función a cada review
df[["num_sentences", "avg_sentence_length"]] = df["review"].apply(longitud_oraciones)

In [ ]:
#Vemos la cantidad de oraciones en cada review
df[["num_sentences"]]

In [ ]:
#Promedio de oraciones en las reviews
df[["num_sentences"]].mean()

In [ ]:
# Promedio de largo de cada oracion (en palabras)
df['avg_sentence_length']

In [ ]:
plt.figure(figsize=(10, 5))
sns.histplot(df["avg_sentence_length"], bins=30, kde=True)
plt.title("Distribución de la longitud promedio de las oraciones")
plt.xlabel("Palabras por oración")
plt.ylabel("Frecuencia")
plt.show()

Vemos que la moda se encuentra entre las 15 y 20 palabras por oración.

####**Frecuencia de Palabras**

In [ ]:
# Se concatena todas las reviews en un texto

text = " ".join(df["review"].dropna().astype(str))

# Tokenizar
tokens = word_tokenize(text.lower())  # pasar a minusculas

# Quitar puntuaciones, numeros y stopwords
stop_words = set(stopwords.words("english"))
tokens_cleaned = [
    word for word in tokens
    if word.isalpha() and word not in stop_words
    ]

In [ ]:
# Contar frecuencia
freq_palabras = Counter(tokens_cleaned)

top_palabras = freq_palabras.most_common(40)

print(top_palabras)

In [ ]:
# se convierte la frecuencia de las palabras a un DF para graficar
df_frec = pd.DataFrame(top_palabras, columns=["word", "frequency"])

plt.figure(figsize=(12, 6))
sns.barplot(data=df_frec, x="frequency", y="word", palette="Blues_d")
plt.title("Top 40 palabras mas frecuentes en las reviews")
plt.xlabel("Frecuencia")
plt.ylabel("Palabra")
plt.tight_layout()
plt.show()

Vemos el top de palabras mas usadas: **Flight, airline, service y time** de las mas comunes entendiblemente. Luego hay algunas interesantes, **hours**, **never, delayed, return, another** , que parece que se refieren a los problemas que son explayados en las reviews.

####**Partes del Discurso (POS)**

In [ ]:
tagged_tokens = pos_tag(tokens)
# Agrupar por tipo
pos_counts = Counter(tag for word, tag in tagged_tokens)

print(pos_counts.most_common(20))

In [ ]:
pos_df = pd.DataFrame(pos_counts.items(), columns=["POS", "Count"])
pos_df = pos_df.sort_values(by="Count", ascending=False).head(10)

plt.figure(figsize=(10, 6))
sns.barplot(data=pos_df, x="Count", y="POS", palette="viridis")
plt.title("Top 10 POS tags en reviews")
plt.xlabel("Frecuencia")
plt.ylabel("Etiqueta POS")
plt.show()

Vemos que los tipos de palabras mas comunes son los sustantivos en primer lugar, luego los articulos o preposiciones, luego los adjetivos y luego los verbos en pasado.

####**Distribución de Longitud de Palabras**

In [ ]:
word_lengths = [len(word) for word in tokens_cleaned]

plt.figure(figsize=(10, 6))
sns.histplot(word_lengths, bins=range(1, 20), kde=False)
plt.title("Distribución de longitud de palabras")
plt.xlabel("Número de letras")
plt.ylabel("Frecuencia")
plt.show()

####**Análisis de N-gramas**

In [ ]:
# Bigramas
bigramas = list(ngrams(tokens_cleaned, 2))
bigramas_freq = Counter(bigramas).most_common(20)

# Trigramas
trigramas = list(ngrams(tokens_cleaned, 3))
trigramas_freq = Counter(trigramas).most_common(20)

print("Top 10 Bigramas:")
for par in bigramas_freq[:10]:
    print(par)

print("\nTop 10 Trigramas:")
for trio in trigramas_freq[:10]:
    print(trio)


Aquí podemos ver los bigramas y trigamas que mas frecuencia tienen en las reviews.

Notamos que en este caso los bigramas mas comunes suelen ser sustantivos, y los trigramas mas comunes suelen ser descriptivos de la experiencia (por lo visto mayormente negativas).

####**Análisis de la Diversidad Léxica**

La diversidad lexica seria la cantidad de palabras unicas dividido la cantidad de palabras totales, para analizar que tan variado es el vocabulario, o si suelen ser palabras comunes y repetidas.

In [ ]:
palabras_totales = len(tokens_cleaned)
palabras_unicas = len(set(tokens_cleaned))
diversidad_lexica = palabras_unicas / palabras_totales

print(f"Diversidad léxica: {diversidad_lexica:.3f}")

La diversidad de 0.020 es muy baja, por lo que suelen ser palabras comunes y repetidas.

A partir de un valor de 0.3 suele ser mas variado, y de mas de 0.7 es muy variado.

####**Visualización de Palabras con Word Cloud**

In [ ]:
wordcloud = WordCloud(width=1000, height=500, background_color="white").generate(" ".join(tokens_cleaned))

plt.figure(figsize=(15, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.title("Nube de palabras de las reviews")
plt.show()

####**Conclusiones del Analisis exploratorio**

Podemos concluir a partir de este analisis, lo siguiente

*   La longitud promedio de las oraciones de las reviews ronda entre **15 y 20 palabras.**
*   Un promedio de 8.33 oraciones por review.

*   Hay bastantes palabras utilizadas muchas veces, pero pocas utilizadas muchisimas veces: ('flight', 42353), ('airline', 12761), ('service', 12697), ('time', 12305).
*   Los **bigramas** mas utilizados, son menciones: (('customer', 'service'), 3984)
(('cabin', 'crew'), 2463)
(('business', 'class'), 2294)


*   Y los **trigramas** mas utilziados son mas descriptivos: (('worst', 'airline','ever'), 462)
(('flight', 'delayed', 'hours'), 339)
(('worst', 'customer', 'service'), 214)
(('cabin', 'crew', 'friendly'), 209)

    Estos muestran aspectos clave que los pasajeros valoran o critican.



*   Se pueden ver temas recurrentes, las reviews se centran mayormente en servicio, comodidad y puntualidad.

### **Análisis Sintáctico**

Podemos ver las relaciones sintacticas en las frases, el tipo de palabra y su relacion con otras.

In [ ]:
review = df['review'].iloc[422]
doc = nlp(review)

# Mostrar estructura de las frases
for token in doc:
    print(f"{token.text:<15} {token.pos_:<10} {token.dep_:<15} {token.head.text}")

###**Análisis Semántico**

Por ejemplo de una review positiva:

In [ ]:
print(df["review"].iloc[0])

In [ ]:
review_pos = df['review'].iloc[0]
blob = TextBlob(review_pos)
print(f"Polaridad: {blob.sentiment.polarity}, Subjetividad: {blob.sentiment.subjectivity}")

La polaridad (de -1.0 a +1.0) indica que tan negativa (-1.0) o positiva (+1.0) es. Y la subjetividad (de 0 a 1) indica la objetividad (0.0) a subjetividad (1.0)

Un ejemplo de review negativa:

In [ ]:
print(df["review"].iloc[18293])

In [ ]:
review_neg = df['review'].iloc[18293]
blob = TextBlob(review_neg)
print(f"Polaridad: {blob.sentiment.polarity}, Subjetividad: {blob.sentiment.subjectivity}")

En este caso la review siendo la review negativa da una polaridad de -0.38.

###**Codificación de texto a vectores**

####Tf-IDF

In [ ]:
reviews = df['review'].dropna().astype(str)

# Tokenizer para mantener palabras alfabeticas
def clean_tokenizer(text):
    return re.findall(r'\b[a-zA-Z]{2,}\b', text.lower())

# se instancia el TF-IDF
tfidf = TfidfVectorizer(tokenizer=clean_tokenizer, stop_words='english', max_df=0.95, min_df=5)

# se aplica al corpus
X_tfidf = tfidf.fit_transform(reviews)

tfidf_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())

print(tfidf_df.head())

In [ ]:
# Puntajes TF-IDF por palabra en todo el corpus
puntajes_tf = tfidf_df.sum().sort_values(ascending=False)


plt.figure(figsize=(10, 6))
puntajes_tf.head(20).plot(kind='bar')
plt.title("Top 20 Palabras con mayor peso TF-IDF")
plt.ylabel("TF-IDF Score")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

El TF-IDF evalua el peso de las palabras en el contexto en el que están. Muchas son coincidentes con las palabras mas utilziadas

####Word Embendings

#**Feature Selection**

**Selección de variable objetivo y variables independientes**

Utilizando Overall_rating como variable target. En primera instancia probaría como positivo un valor mayor de 5, y negativo como menor que 5 inclusive.

In [ ]:
df = df.dropna(subset=['review', 'overall_rating'])
df['overall_rating'] = df['overall_rating'].astype(float)
#Se define la calificacion a binario
df['label'] = df['overall_rating'].apply(lambda x: 1 if x >= 5 else 0)

# variables x
X = tfidf.fit_transform(df['review'])

# target
y = df['label']

In [ ]:
df['label'].value_counts()

La cantidad de reviews negativas quedo en 16836 y la cantidad de positivas en 6215. Un desbalance que puede afectar al modelo.

# **Modelos**

**División de datos en conjuntos de entrenamiento y prueba**

In [ ]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

###Balanceo de clases

Al tener una diferencia tan grande entre reviews positivas y negativas hay que hacer un resampling para balancear la cantidad de reviews positivas y negativas, asi el modelo se entrena correctamente.

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

##Regresion Logistica

In [ ]:
# Regresion logistica
clf = LogisticRegression(C=1, class_weight='balanced')
clf.fit(X_resampled, y_resampled)

**Evaluación del rendimiento del modelo**

In [ ]:
# Evaluación
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

**Predicción con conjunto de prueba**

In [ ]:
review_nueva = ["The flight was comfortable and the staff were very friendly."]

# Convertir la review a vector
X_new = tfidf.transform(review_nueva)

# Prediccion
prediction = clf.predict(X_new)
proba = clf.predict_proba(X_new)

print("Predicción:", "Positiva" if prediction[0] == 1 else "Negativa")
print("Probabilidad:", proba[0])

Vemos que la predicción dio una probabilidad de 0.03 de que sea negativa, y una probabilidad de 0.96 de que sea positiva.

In [ ]:
test_reviews = [
    "The flight was delayed and the food was terrible.",
    "Fantastic service and very smooth check-in process.",
    "Seats were okay, nothing special.",
    "Terrible flight, rude staff, and broken seats.",
    "One of the best flights I’ve had in years!"
]

# Transformar con el mismo vectorizador TF-IDF
X_test_reviews = tfidf.transform(test_reviews)

# Obtener predicciones y probabilidades
predictions = clf.predict(X_test_reviews)
probabilities = clf.predict_proba(X_test_reviews)

# Mostrar resultados
for review, pred, proba in zip(test_reviews, predictions, probabilities):
    sentiment = "Positiva" if pred == 1 else "Negativa"
    confidence = proba[1] if pred == 1 else proba[0]

    print(f"📝 Review: {review}")
    print(f"🔮 Predicción: {sentiment} (Confianza: {confidence:.2f})")
    print("-" * 60)

**Conclusión sobre el modelado y las metricas**

La regresión logistica dio un resultado bastante aceptable en primera instancia, con una precisión, 0.8 aprox
Pero con un recall bajo (58%), el modelo no identifica bien las reviews positivas.

F1-score general: aceptable, pero mejorable.

##Random Forest Classifier

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_resampled, y_resampled)

In [ ]:
# Evaluación
y_pred = rf_clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
review_nueva = ["The flight was comfortable and the staff were very friendly."]

# Convertir la review a vector
X_new = tfidf.transform(review_nueva)

# Prediccion
prediction = rf_clf.predict(X_new)
proba = rf_clf.predict_proba(X_new)

print("Predicción:", "Positiva" if prediction[0] == 1 else "Negativa")
print("Probabilidad:", proba[0])

##XGBoost + TF-IDF

In [ ]:
xgb = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    n_jobs=-1,
    random_state=42
)

# Grid de parametros
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1],
    'scale_pos_weight': [1, len(y[y == 0]) / len(y[y == 1])]
}

# Crear GridSearch
grid = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='f1',
    cv=3,
    verbose=2
)

# Entrenamiento
grid.fit(X_resampled, y_resampled)

In [ ]:
print("Mejores parámetros:", grid.best_params_)

best_model = grid.best_estimator_

# Evaluar
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
new_reviews = [
    "Amazing flight experience, very smooth and clean.",
    "Worst flight I've ever had, very rude staff."
]

X_new = tfidf.transform(new_reviews)
preds = best_model.predict(X_new)
probas = best_model.predict_proba(X_new)

for review, pred, proba in zip(new_reviews, preds, probas):
    label = "Positiva" if pred == 1 else "Negativa"
    print(f"📝 {review}")
    print(f"🔮 {label} (Confianza: {proba[pred]:.2f})\n")

#**Optimización de modelos**

In [ ]:
#Optimizacion de Regresion logistica
params = {
    'C': [0.1, 1, 10],
    'class_weight': [None, 'balanced']
}

grid = GridSearchCV(LogisticRegression(), params, cv=5, scoring='f1',verbose=2)
grid.fit(X_resampled, y_resampled)

print("Best params:", grid.best_params_)

In [ ]:
# Optimizacion de Random Forest
param_grid = {
    'n_estimators': [100, 150],           # Cantidad de árboles
    'max_depth': [None, 10, 20],      # Profundidad máxima de los árboles
    'min_samples_split': [ 5, 10],      # Mínimo de muestras para dividir un nodo
    'min_samples_leaf': [2, 4],        # Mínimo de muestras en una hoja
    'max_features': ['sqrt'],     # Qué proporción de features usar
    'class_weight': [None, 'balanced']    # Para manejar el desbalance de clases
}


grid_search = GridSearchCV(
    estimator=rf_clf,
    param_grid=param_grid,
    cv=3,
    n_jobs=-1,           # Usa todos los núcleos disponibles
    scoring='f1',        # Métrica para evaluar el balance precisión/recall
    verbose=2
)
grid_search.fit(X_resampled, y_resampled)

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor puntaje F1:", grid_search.best_score_)

# **Conclusiones Finales**

Se puede predecir con cierta precisión el sentimiento de las nuevas reviews que van ingresando a las plataformas, con el fin de categorizarlas y obtener así un insight mayor, sobre la experiencia de usuario y los puntos clave que son valorados o criticados.